In [1]:
import pandas as pd
import pickle
# ^^^ pyforest auto-imports - don't write above this line
import xgboost as xgb
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)

In [2]:
df=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
df1=df[["tenure", "MonthlyCharges", "TotalCharges", "OnlineSecurity", "Contract", "InternetService", "TechSupport", "Churn"]]
df1.head()

,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,Contract,InternetService,TechSupport,Churn
0,1,29.85,29.85,No,Month-to-month,DSL,No,No
1,34,56.95,1889.5,Yes,One year,DSL,No,No
2,2,53.85,108.15,Yes,Month-to-month,DSL,No,Yes
3,45,42.30,1840.75,Yes,One year,DSL,Yes,No
4,2,70.70,151.65,No,Month-to-month,Fiber optic,No,Yes


In [5]:
df1.dtypes

tenure               int64
MonthlyCharges     float64
TotalCharges        object
OnlineSecurity      object
Contract            object
InternetService     object
TechSupport         object
Churn               object
dtype: object

In [6]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges, errors='coerce')

In [7]:
df1.isnull().sum()

tenure              0
MonthlyCharges      0
TotalCharges       11
OnlineSecurity      0
Contract            0
InternetService     0
TechSupport         0
Churn               0
dtype: int64

In [8]:
df1.dropna(inplace=True)

In [9]:
df1.head()

,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,Contract,InternetService,TechSupport,Churn
0,1,29.85,29.85,No,Month-to-month,DSL,No,No
1,34,56.95,1889.50,Yes,One year,DSL,No,No
2,2,53.85,108.15,Yes,Month-to-month,DSL,No,Yes
3,45,42.30,1840.75,Yes,One year,DSL,Yes,No
4,2,70.70,151.65,No,Month-to-month,Fiber optic,No,Yes


In [10]:
df_final=df1.drop("Churn", axis=1)

In [11]:
pickle.dump(df_final, open("df_final", "wb"))

<IPython.core.display.Javascript object>

In [12]:
df1.OnlineSecurity.value_counts()

No                     3497
Yes                    2015
No internet service    1520
Name: OnlineSecurity, dtype: int64

In [13]:
df1.tenure.describe()

count    7032.000000
mean       32.421786
std        24.545260
min         1.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

In [14]:
df1.MonthlyCharges.describe()

count    7032.000000
mean       64.798208
std        30.085974
min        18.250000
25%        35.587500
50%        70.350000
75%        89.862500
max       118.750000
Name: MonthlyCharges, dtype: float64

In [15]:
df1.TotalCharges.describe()

count    7032.000000
mean     2283.300441
std      2266.771362
min        18.800000
25%       401.450000
50%      1397.475000
75%      3794.737500
max      8684.800000
Name: TotalCharges, dtype: float64

In [16]:
df1.Contract.value_counts()

Month-to-month    3875
Two year          1685
One year          1472
Name: Contract, dtype: int64

In [17]:
df1.InternetService.value_counts()

Fiber optic    3096
DSL            2416
No             1520
Name: InternetService, dtype: int64

In [18]:
df1.TechSupport.value_counts()

No                     3472
Yes                    2040
No internet service    1520
Name: TechSupport, dtype: int64

In [19]:
df1["Churn"]=df1["Churn"].replace(["Yes", "No"], [1,0])
df1["Churn"].head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

In [20]:
df2=pd.get_dummies(df1)

In [21]:
X=df2.drop("Churn", axis=1)
y=df2.Churn

In [22]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=41, stratify=y)

In [23]:
X.columns

Index(['tenure', 'MonthlyCharges', 'TotalCharges', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes'],
      dtype='object')

In [24]:
deploy_df=X.copy()

In [25]:
deploy_df.head()

,tenure,MonthlyCharges,TotalCharges,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,InternetService_DSL,InternetService_Fiber optic,InternetService_No,TechSupport_No,TechSupport_No internet service,TechSupport_Yes
0,1,29.85,29.85,1,0,0,1,0,0,1,0,0,1,0,0
1,34,56.95,1889.50,0,0,1,0,1,0,1,0,0,1,0,0
2,2,53.85,108.15,0,0,1,1,0,0,1,0,0,1,0,0
3,45,42.30,1840.75,0,0,1,0,1,0,1,0,0,0,0,1
4,2,70.70,151.65,1,0,0,1,0,0,0,1,0,1,0,0


In [26]:
#pickle.dump(deploy_df, open("deploy_df", "wb"))

In [27]:
from xgboost import XGBClassifier

In [28]:
import pickle

In [33]:
xgb_final=XGBClassifier(learning_rate= 0.1,
                                max_depth= 6,
                                n_estimators= 300,
                                subsample= 1).fit(X, y)

In [34]:
y_pred=xgb_final.predict(X_test)
confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92      1549
           1       0.84      0.71      0.77       561

    accuracy                           0.89      2110
   macro avg       0.87      0.83      0.85      2110
weighted avg       0.88      0.89      0.88      2110



In [35]:
pickle.dump(xgb_final, open("my_model_2", "wb"))

In [1]:
my_model=pickle.load(open("my_model", "rb"))

<IPython.core.display.Javascript object>

In [91]:
#sample=df2[55:60].drop("Churn", axis=1)

In [92]:
#sample.head()

Index(['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes'],
      dtype='object')

In [170]:
my_model.predict(sample)

array([1, 0, 0, 0, 0], dtype=int64)

In [171]:
df3[55:60].Churn

55    1
56    0
57    0
58    0
59    0
Name: Churn, dtype: int64

In [2]:
features=['tenure', 'MonthlyCharges', 'TotalCharges',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes']

In [3]:
pickle.dump(features, open("features", "wb"))

In [7]:
def single_customer(tenure, MonthlyCharges, TotalCharges, OnlineSecurity, Contract, InternetService, TechSupport):
    my_dict={"tenure":tenure, "OnlineSecurity":OnlineSecurity, "Contract":Contract, "TotalCharges":TotalCharges, "InternetService":InternetService, TechSupport:"TechSupport", "MonthlyCharges":MonthlyCharges}
    
    columns = ['tenure', 'MonthlyCharges', 'TotalCharges',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes']
    
    df = pd.DataFrame.from_dict([my_dict], orient="columns")
    X = pd.get_dummies(df).reindex(columns=columns, fill_value=0)
    result = my_model.predict(X)[0]
    return result

In [8]:
single_customer(2, 53.85, 108.15, "Yes","Month-to-month","DSL","No")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1

In [ ]:
1	29.85	29.85	No	Month-to-month	DSL	No	No

In [ ]:
2	53.85	108.15	Yes	Month-to-month	DSL	No	Yes

In [65]:
df1["Churn Prob"]=X.apply(single_customer)

TypeError: single_customer() missing 6 required positional arguments: 'MonthlyCharges', 'TotalCharges', 'OnlineSecurity', 'Contract', 'InternetService', and 'TechSupport'